In [1]:
# keep TF 2.17 => requires numpy < 2.0
!pip -q install --upgrade pip
!pip -q install --no-cache-dir \
  numpy==1.26.4 scipy==1.13.1 \
  pandas==2.2.2 scikit-learn==1.5.2 \
  transformers==4.44.2 tensorflow==2.17.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 49.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, but you have scikit-learn 1.5.2 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.8 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.
tensorflow-decision-forests 1.12.0 requires tensorflow==2.19.0, but you have tensorflow 2

In [ ]:
import os, sys
os.kill(os.getpid(), 9)


In [1]:
import os, gc, numpy as np, pandas as pd, tensorflow as tf
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from transformers import DistilBertTokenizerFast, TFDistilBertModel

print("NumPy:", np.__version__)
print("Pandas:", pd.__version__)
print("scikit-learn:", __import__("sklearn").__version__)
print("Transformers:", __import__("transformers").__version__)
print("TF:", tf.__version__)
print("GPUs:", tf.config.list_physical_devices("GPU"))

MAX_LEN = 64
MAX_TRAIN_ROWS = 120_000
SCALE_PRICE = 100_000.0


NumPy: 1.26.4
Pandas: 2.2.2
scikit-learn: 1.5.2
Transformers: 4.44.2
TF: 2.17.0
GPUs: []


In [2]:
# ============================================
# 1) LOAD DATA (20% subset file)
# ============================================
csv_paths = ["/content/df_subset_20.csv", "/mnt/data/df_subset_20.csv"]
csv_path = next((p for p in csv_paths if Path(p).exists()), None)
if csv_path is None:
    from google.colab import files
    uploaded = files.upload()   # upload df_subset_20.csv
    csv_path = list(uploaded.keys())[0]

df = pd.read_csv(csv_path)

# Expected columns (case-insensitive remap)
cols = {c.lower(): c for c in df.columns}
def col(name): return cols.get(name.lower(), name)

df = df.rename(columns={
    col("title"): "title",
    col("title_clean"): "title_clean",
    col("category"): "category",
    col("category_encoded"): "category_encoded",
    col("price"): "price",
    col("isBestSeller"): "isBestSeller",
    col("boughtInLastMonth"): "boughtInLastMonth",
})

# Minimal cleaning
if "title_clean" not in df.columns:
    df["title_clean"] = df["title"].astype(str).str.lower().str.strip()
df["category"] = df["category"].astype(str).str.strip()
df["price"] = pd.to_numeric(df["price"], errors="coerce")
df["isBestSeller"] = pd.to_numeric(df["isBestSeller"], errors="coerce").fillna(0).astype(int)
if "boughtInLastMonth" in df.columns:
    df["boughtInLastMonth"] = pd.to_numeric(df["boughtInLastMonth"], errors="coerce")

df = df.dropna(subset=["title_clean","category","price"]).reset_index(drop=True)

print("Rows after cleaning:", len(df))
df.head(3)


Rows after cleaning: 533293


,title,stars,reviews,price,category,isBestSeller,boughtInLastMonth,category_encoded,title_clean
0,"Power Tool Organizer Shelf, Heavy Duty Drill H...",0.92,0.000307,0.001296,Home Storage Organization,0,0.0025,118,"power tool organizer shelf, heavy duty drill h..."
1,"Disney Girls Baseball Cap, Frozen Elsa & Anna ...",0.94,0.000379,0.001198,Sporting Apparel,0,0.0000,225,"disney girls baseball cap, frozen elsa & anna ..."
2,Micro OTG Card Reader High-Speed USB Universal...,0.00,0.000000,0.000000,Memory Card Accessories,0,0.0000,163,micro otg card reader high-speed usb universal...


In [21]:
# ============================================
# 2) KPIs
# ============================================
avg_price_by_cat = df.groupby('category')['price'].mean().sort_values(ascending=False)
avg_sales_by_cat = df.groupby('category')['boughtInLastMonth'].mean().sort_values(ascending=False)
bestseller_ratio = (df.groupby('category')['isBestSeller'].mean() * 100).sort_values(ascending=False)

kpi = pd.DataFrame({
    "AvgPrice_scaled": avg_price_by_cat,
    "AvgMonthlySales_scaled": avg_sales_by_cat,
    "BestSellerRatio_%": bestseller_ratio
})
kpi["AvgPrice_USD"] = (kpi["AvgPrice_scaled"] * SCALE_PRICE).round(2)
print("=== KPI preview ===")
kpi.head(10)


=== KPI preview ===


,AvgPrice_scaled,AvgMonthlySales_scaled,BestSellerRatio_%,AvgPrice_USD
category,,,,
3D Printing Scanning,0.001809,0.000045,0.000000,180.92
Abrasive Finishing Products,0.001016,0.000107,0.214592,101.60
"Action Figures, Maquettes Busts",0.000988,0.000308,0.000000,98.79
Action Sports,0.001457,0.000179,1.163919,145.70
Air Freshener Supplies,0.000976,0.000603,0.106724,97.58
Arts Crafts Supplies,0.000562,0.000828,0.438169,56.22
Artwork,0.000693,0.000105,0.000000,69.27
Audio/Video Receivers Amplifiers,0.003215,0.000002,0.080257,321.55
Automotive,0.001326,0.002365,5.824742,132.56


# ***My notes for improvement***

Our KPI table shows that BestSellerRatio_% is almost zero for most categories, indicating a severe data imbalance and low positive rate for the ‘bestseller’ label. This imbalance causes the model to overfit on the dominant class (non-bestsellers) and fail to generalize, despite showing high accuracy. To fix this, we should recompute bestseller ratios on smaller groups, apply class weighting or focal loss, upsample positive cases, and include additional popularity-related features (like review count or rating). These changes will help the model capture real bestseller patterns instead of just predicting the majority class.

In [20]:
# ============================================
# 3) AGILE SPRINT SAMPLE (use all rows in file,
#    but cap to keep runtime small)
# ============================================
if len(df) > MAX_TRAIN_ROWS:
    df_small = df.sample(n=MAX_TRAIN_ROWS, random_state=42).reset_index(drop=True)
else:
    df_small = df.copy()

print("Training rows (sprint):", len(df_small))

# Build/ensure category_encoded
if "category_encoded" not in df_small.columns:
    # One-time fit over sprint slice
    ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)
    cats_2d = df_small[["category"]]
    _ = ohe.fit(cats_2d)
    cat2id = {cat: i for i, cat in enumerate(ohe.categories_[0])}
    df_small["category_encoded"] = df_small["category"].map(cat2id).fillna(0).astype(int)
else:
    # Still build mapping for inference convenience
    uniq = df_small["category"].dropna().unique().tolist()
    cat2id = {c:i for i,c in enumerate(sorted(uniq))}
    # if provided encodings exist, prefer them
    if df_small["category_encoded"].nunique() >= len(cat2id)*0.8:
        id_series = df_small["category_encoded"].astype(int)
        # build reverse safely
        # we’ll rebuild at inference using df mapping anyway
    else:
        df_small["category_encoded"] = df_small["category"].map(cat2id).fillna(0).astype(int)

num_categories = int(df_small["category_encoded"].max()) + 1
print("num_categories:", num_categories)


Training rows (sprint): 120000
num_categories: 266


In [5]:
# ============================================
# 4) TEXT PREP: TF-IDF (light) + DistilBERT (frozen)
# ============================================
# TF-IDF features (optionally concatenate later if you want)
tfidf = TfidfVectorizer(max_features=3000, ngram_range=(1,2), min_df=3)
X_tfidf = tfidf.fit_transform(df_small["title_clean"]).astype("float32")

# DistilBERT tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
enc = tokenizer(list(df_small["title_clean"]),
                padding="max_length", truncation=True, max_length=MAX_LEN,
                return_tensors="np")

input_ids_full = enc["input_ids"]
attn_masks_full = enc["attention_mask"]
cat_ids_full = df_small["category_encoded"].values.reshape(-1,1).astype("int32")

y_flag_full  = df_small["isBestSeller"].values.astype("float32")
y_price_full = df_small["price"].values.astype("float32")

print("Shapes:", input_ids_full.shape, attn_masks_full.shape, cat_ids_full.shape)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Shapes: (120000, 64) (120000, 64) (120000, 1)


In [6]:
# ============================================
# 5) TRAIN/VAL SPLIT
# ============================================
(X_ids_tr, X_ids_te,
 X_mask_tr, X_mask_te,
 X_cat_tr, X_cat_te,
 y_flag_tr, y_flag_te,
 y_price_tr, y_price_te) = train_test_split(
    input_ids_full, attn_masks_full, cat_ids_full,
    y_flag_full, y_price_full,
    test_size=0.10, random_state=42, stratify=(y_flag_full>0.5)
)

# Report imbalance
pos_rate = y_flag_tr.mean()
print(f"Train positive rate (isBestSeller=1): {pos_rate:.4%}")


Train positive rate (isBestSeller=1): 0.6213%


In [7]:
# ============================================
# 6) MODEL: DistilBERT (frozen) + Category Embedding + TF-IDF
#     → shared trunk → two heads (bestseller flag, price)
# ============================================
import tensorflow as tf
from tensorflow.keras import layers, Model
from transformers import TFDistilBertModel
import scipy.sparse as sp

# ---- Inputs (assumes MAX_LEN, num_categories, X_tfidf already defined)
ids_in = layers.Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_ids")
msk_in = layers.Input(shape=(MAX_LEN,), dtype=tf.int32, name="attention_mask")
cat_in = layers.Input(shape=(1,),       dtype=tf.int32, name="category_id")
X_tfidf_dim = int(X_tfidf.shape[1])
tfidf_in = layers.Input(shape=(X_tfidf_dim,), dtype=tf.float32, name="tfidf_in")

# ---- DistilBERT backbone (frozen)
bert = TFDistilBertModel.from_pretrained("distilbert-base-uncased")
bert.trainable = False
H = int(getattr(bert.config, "hidden_size", getattr(bert.config, "dim", 768)))  # 768 for DistilBERT

# Keras-3 safe wrapper: accept **kwargs and declare output_shape
def _bert_last_hidden_state(inputs, **kwargs):
    outputs = bert(inputs, training=False)  # accepts dict
    return outputs.last_hidden_state if hasattr(outputs, "last_hidden_state") else outputs[0]

seq_out = layers.Lambda(
    _bert_last_hidden_state, name="bert", output_shape=(MAX_LEN, H)
)({"input_ids": ids_in, "attention_mask": msk_in})        # (B, L, H)

cls_vec  = seq_out[:, 0, :]                                # (B, H)
text_vec = layers.Dense(128, activation="relu", name="text_dense")(cls_vec)

# ---- Category embedding branch
cat_emb = layers.Embedding(input_dim=max(int(num_categories), 2),
                           output_dim=16, name="cat_emb")(cat_in)
cat_vec = layers.Flatten(name="cat_flat")(cat_emb)

# ---- TF-IDF branch
tfidf_vec = layers.Dense(128, activation="relu", name="tfidf_dense")(tfidf_in)

# ---- Fuse + heads
fused = layers.Concatenate(name="fuse")([text_vec, cat_vec, tfidf_vec])
h = layers.Dense(128, activation="relu", name="dense_128")(fused)
h = layers.Dense(64,  activation="relu", name="dense_64")(h)

flag_out  = layers.Dense(1, activation="sigmoid", name="flag_output")(h)   # bestseller prob
price_out = layers.Dense(1, activation="linear",  name="price_output")(h)  # price regression

model = Model(
    inputs=[ids_in, msk_in, cat_in, tfidf_in],
    outputs=[flag_out, price_out],
    name="mtl_distilbert_tfidf"
)

# ---- Compile (use MAE class, not function)
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss={"flag_output": "binary_crossentropy", "price_output": "mse"},
    loss_weights={"flag_output": 1.5, "price_output": 0.5},
    metrics={
        "flag_output": [tf.keras.metrics.AUC(name="auc"),
                        tf.keras.metrics.BinaryAccuracy(name="acc")],
        "price_output": [tf.keras.metrics.MeanAbsoluteError(name="mae")],
    },
)

model.summary()

# (If your TF-IDF matrix is sparse, densify once before fit)
X_tfidf_dense = X_tfidf.toarray().astype("float32") if sp.issparse(X_tfidf) else X_tfidf.astype("float32")
# When fitting, feed inputs in this order:
# [input_ids, attention_mask, category_id, X_tfidf_dense]


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Model: "mtl_distilbert_tfidf"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ attention_mask      │ (None, 64)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_ids           │ (None, 64)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bert (Lambda)       │ (None, 64, 768)   │          0 │ attention_mask[0… │
│                     │                   │            │ input_ids[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ category_id         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item (GetItem)  │ (None, 768)       │          0 │ bert[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cat_emb (Embedding) │ (None, 1, 16)     │      4,256 │ category_id[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tfidf_in            │ (None, 3000)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ text_dense (Dense)  │ (None, 128)       │     98,432 │ get_item[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cat_flat (Flatten)  │ (None, 16)        │          0 │ cat_emb[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tfidf_dense (Dense) │ (None, 128)       │    384,128 │ tfidf_in[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fuse (Concatenate)  │ (None, 272)       │          0 │ text_dense[0][0], │
│                     │                   │            │ cat_flat[0][0],   │
│                     │                   │            │ tfidf_dense[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_128 (Dense)   │ (None, 128)       │     34,944 │ fuse[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_64 (Dense)    │ (None, 64)        │      8,256 │ dense_128[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flag_output (Dense) │ (None, 1)         │         65 │ dense_64[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ price_output        │ (None, 1)         │         65 │ dense_64[0][0]    │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 530,146 (2.02 MB)

 Trainable params: 530,146 (2.02 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
# ============================================
# 7) TRAIN — tf.data zip (no TypeSpec errors)
# ============================================
import numpy as np, tensorflow as tf
from collections import Counter
from scipy import sparse as sp

# 1) Dense TF-IDF once + consistent dtypes
X_tfidf_dense = X_tfidf.toarray().astype("float32") if sp.issparse(X_tfidf) else X_tfidf.astype("float32")
n_tr, n_te = len(X_ids_tr), len(X_ids_te)
X_tfidf_tr = X_tfidf_dense[:n_tr]
X_tfidf_te = X_tfidf_dense[n_tr:n_tr+n_te]

X_ids_tr  = np.asarray(X_ids_tr,  dtype="int32")
X_mask_tr = np.asarray(X_mask_tr, dtype="int32")
X_cat_tr  = np.asarray(X_cat_tr,  dtype="int32")          # shape (N,) or (N,1) both OK
X_ids_te  = np.asarray(X_ids_te,  dtype="int32")
X_mask_te = np.asarray(X_mask_te, dtype="int32")
X_cat_te  = np.asarray(X_cat_te,  dtype="int32")

y_flag_tr  = np.asarray(y_flag_tr,  dtype="float32")
y_price_tr = np.asarray(y_price_tr, dtype="float32")
y_flag_te  = np.asarray(y_flag_te,  dtype="float32")
y_price_te = np.asarray(y_price_te, dtype="float32")

# 2) Per-output sample weights
ctr = Counter(y_flag_tr.astype(int))
w0 = len(y_flag_tr) / (2.0 * max(ctr.get(0, 1), 1))
w1 = len(y_flag_tr) / (2.0 * max(ctr.get(1, 1), 1))
w_flag_tr  = np.where(y_flag_tr.astype(int) == 1, w1, w0).astype("float32")
w_flag_te  = np.where(y_flag_te.astype(int) == 1, w1, w0).astype("float32")
w_price_tr = np.ones_like(y_price_tr, dtype="float32")
w_price_te = np.ones_like(y_price_te, dtype="float32")
print("class_weight(flag)~", {0: w0, 1: w1})

# 3) Build datasets with TUPLES and ZIP (avoids non-rectangular list issue)
def make_ds(X_ids, X_mask, X_cat, X_tf, y_flag, y_price, w_flag, w_price, batch, shuffle=True):
    ds_in  = tf.data.Dataset.from_tensor_slices((X_ids, X_mask, X_cat, X_tf))      # tuple = OK
    ds_out = tf.data.Dataset.from_tensor_slices((y_flag, y_price))
    ds_w   = tf.data.Dataset.from_tensor_slices((w_flag, w_price))
    ds = tf.data.Dataset.zip((ds_in, ds_out, ds_w))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(X_ids), reshuffle_each_iteration=True)
    return ds.batch(batch).prefetch(tf.data.AUTOTUNE)

BATCH = 512 if tf.config.list_physical_devices('GPU') else 64
train_ds = make_ds(X_ids_tr, X_mask_tr, X_cat_tr, X_tfidf_tr,
                   y_flag_tr, y_price_tr, w_flag_tr, w_price_tr, batch=BATCH, shuffle=True)
val_ds   = make_ds(X_ids_te, X_mask_te, X_cat_te, X_tfidf_te,
                   y_flag_te, y_price_te, w_flag_te, w_price_te, batch=BATCH, shuffle=False)

# 4) Train fast
ES = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True)
hist = model.fit(train_ds, validation_data=val_ds, epochs=4, callbacks=[ES], verbose=1)


class_weight(flag)~ {0: 0.5031259025985522, 1: 80.4769001490313}
Epoch 1/4
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 2144s 1s/step - flag_output_acc: 0.7370 - flag_output_auc: 0.7500 - flag_output_loss: 0.5910 - loss: 0.8874 - price_output_loss: 0.0018 - price_output_mae: 0.0311 - val_flag_output_acc: 0.8709 - val_flag_output_auc: 0.8057 - val_flag_output_loss: 0.5951 - val_loss: 0.8948 - val_price_output_loss: 0.0010 - val_price_output_mae: 0.0243
Epoch 2/4
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 2085s 1s/step - flag_output_acc: 0.8318 - flag_output_auc: 0.9234 - flag_output_loss: 0.3405 - loss: 0.5114 - price_output_loss: 0.0013 - price_output_mae: 0.0262 - val_flag_output_acc: 0.8733 - val_flag_output_auc: 0.7547 - val_flag_output_loss: 0.8969 - val_loss: 1.3486 - val_price_output_loss: 7.3260e-04 - val_price_output_mae: 0.0221
Epoch 3/4
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 2050s 1s/step - flag_output_acc: 0.8887 - flag_output_auc: 0.9751 - flag_output_loss: 0.1983 - loss: 0.2977 - price_output_loss: 4.8450e

In [13]:
# ============================================
# 8) VALIDATION METRICS + THRESHOLD TUNING
# ============================================
from sklearn.metrics import precision_recall_curve, f1_score, mean_absolute_error

pred_flag_val, pred_price_val = model.predict(
    [X_ids_te, X_mask_te, X_cat_te, X_tfidf[df_small.index[len(X_ids_tr):len(X_ids_tr)+len(X_ids_te)]]],
    verbose=0
)
probs = pred_flag_val.ravel()
prec, rec, thr = precision_recall_curve(y_flag_te.astype(int), probs)
f1s = 2*prec*rec/(prec+rec+1e-12)
best_i = np.nanargmax(f1s)
BEST_THR = float(thr[best_i]) if len(thr) else 0.5

f1 = f1_score(y_flag_te.astype(int), (probs >= BEST_THR).astype(int))
mae = mean_absolute_error(y_price_te, pred_price_val.ravel())

print(f"Best threshold (F1): {BEST_THR:.3f} | F1: {f1:.3f}")
print(f"Price MAE (scaled): {mae:.5f}  | MAE in USD: {mae*SCALE_PRICE:,.2f}")


Best threshold (F1): 0.907 | F1: 0.130
Price MAE (scaled): 0.02428  | MAE in USD: 2,428.10


In [18]:
# ============================================
# 9) INFERENCE HELPER (shape-safe)
# ============================================
import numpy as np
from scipy import sparse as sp

# category -> id map (fallback 0 if unseen)
cat2id = dict(zip(df_small["category"], df_small["category_encoded"]))
num_categories = int(max(cat2id.values()) + 1)

def _seq_len_from_model(m):
    # model.inputs[0] is "input_ids" with shape (None, L)
    return int(m.inputs[0].shape[1])

def _tfidf_dim_from_model(m):
    # model.inputs[3] is "tfidf_in" with shape (None, D)
    return int(m.inputs[3].shape[1])

def predict_one(title: str, category: str, thr: float = None):
    """
    Returns (price_usd, bestseller_prob, is_bestseller)
    Works with the 4-input model: [input_ids, attention_mask, category_id, tfidf_in]
    """
    text = str(title).lower().strip()
    seq_len = _seq_len_from_model(model)           # e.g., 64
    tfidf_dim = _tfidf_dim_from_model(model)       # sanity check

    # 1) tokenize to the seq_len the model expects
    enc = tokenizer([text], padding="max_length",
                    truncation=True, max_length=seq_len,
                    return_tensors="np")
    ids = enc["input_ids"]            # shape (1, seq_len)
    msk = enc["attention_mask"]       # shape (1, seq_len)

    # 2) category id (shape (1,1), clipped to valid range)
    cid = int(cat2id.get(category, 0))
    cid = max(0, min(cid, num_categories - 1))
    cat_in = np.array([[cid]], dtype="int32")

    # 3) TF-IDF vector (dense float32, correct width)
    Xt = tfidf.transform([text])
    Xt = Xt.toarray().astype("float32") if sp.issparse(Xt) else Xt.astype("float32")
    if Xt.shape[1] != tfidf_dim:
        # In case a different vectorizer was loaded, pad/truncate to expected D
        if Xt.shape[1] > tfidf_dim:
            Xt = Xt[:, :tfidf_dim]
        else:
            pad = np.zeros((Xt.shape[0], tfidf_dim - Xt.shape[1]), dtype="float32")
            Xt = np.concatenate([Xt, pad], axis=1)

    # 4) predict
    p_flag, p_price = model.predict([ids, msk, cat_in, Xt], verbose=0)
    prob = float(p_flag[0, 0])
    thr = float(BEST_THR) if thr is None else float(thr)
    is_best = prob >= thr

    price_scaled = float(p_price[0, 0])
    price_usd = price_scaled * float(SCALE_PRICE)

    return price_usd, prob, is_best

# ---- DEMO ----
demo_title = ("EZY DOSE Pill Cutter and Splitter with Dispenser, Cuts Pills, Vitamins, Tablets, Stainless Steel Blade, Travel Sized, Purple")
demo_category = "Health Care Products"  # must match a training category label

pred_usd, prob_best, is_best = predict_one(demo_title, demo_category)
print("— DEMO —")
print("Category:", demo_category)
print(f"Bestseller Probability: {prob_best:.3f}")
print("Bestseller?:", "YES ✅" if is_best else "NO ❌")
print(f"Predicted Price: ${pred_usd:,.2f}")



— DEMO —
Category: Health Care Products
Bestseller Probability: 0.233
Bestseller?: NO ❌
Predicted Price: $1,882.74


In [16]:
# ============================================
# 10) SAVE ARTIFACTS (for Jira attachment / reuse)
# ============================================
import joblib, os
os.makedirs("artifacts", exist_ok=True)
model.save("artifacts/mtl_distilbert_tfidf.keras")
joblib.dump(tfidf, "artifacts/tfidf.pkl")
pd.Series(cat2id).to_csv("artifacts/category_to_id.csv", index=True, header=["id"])
print("Saved: artifacts/mtl_distilbert_tfidf.keras, tfidf.pkl, category_to_id.csv")


Saved: artifacts/mtl_distilbert_tfidf.keras, tfidf.pkl, category_to_id.csv


# ***My notes for improvement***

Our model performs very well in predicting product prices but struggles with predicting whether an item is a bestseller. This is mainly due to data imbalance and weak signal in the bestseller labels. The dataset likely has far fewer positive (bestseller) examples compared to negatives, which makes the model biased toward predicting ‘not bestseller.’ Also, bestseller status depends on many hidden factors—like marketing, stock levels, or user engagement—that aren’t fully captured in our data. To improve, we can rebalance the dataset (by oversampling positives or using class weights), try Focal Loss to make the model pay more attention to rare cases, and add richer features such as review count, rating, or price rank within each category. These changes should help improve F1 and recall for bestseller prediction in the next iteration.

Even though our model shows around 97% accuracy, it’s likely overfitting — especially because the bestseller class is very imbalanced. High accuracy here mostly reflects how well the model predicts the majority class (non-bestsellers), not real performance on the minority class. This means the model has learned to memorize patterns from the training data rather than generalizing. The low F1 or recall for bestseller prediction confirms this. To reduce overfitting, we should use stronger regularization (dropout, L2), simplify the dense layers, and ensure we use stratified splits and early stopping. We can also balance the data or use Focal Loss to force the model to learn from the harder, minority samples. After these fixes, validation metrics like PR-AUC or F1 will give a more realistic picture than accuracy alone.